In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import bz2 # To open zipped files
import re # regular expressions
import os
import gc       #for data extraction
import random
from bs4 import BeautifulSoup 
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download("wordnet")
lemmatizer = WordNetLemmatizer()
nltk.download("stopwords") 
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier
import torch
import itertools
import codecs
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import string
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab
import seaborn as sns
from nltk import pos_tag
from gensim.utils import simple_preprocess
nltk.download('cess_esp')  
nltk.download('words')
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package cess_esp to /root/nltk_data...
[nltk_data]   Unzipping corpora/cess_esp.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [2]:
for dirname, _, filenames in os.walk('drive\MyDrive\dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [3]:
print(os.listdir('/content/drive/MyDrive/dataset'))


['test.ft.txt', 'test.ft.txt.bz2', 'train.ft.txt.bz2', 'train.ft.txt', 'archive.zip', 'Cod_dataset_importat.ipynb', 'in_spyder_copy.py', 'in_spider_torch.py', 'train.txt', 'ccb0553edb6abafd20c3dec34ee8099d-master.zip', 'dataframe_dataset.py', 'json.py', '.ipynb_checkpoints', 'graphs', 'ccb0553edb6abafd20c3dec34ee8099d-master', 'in_spyder.py']


In [4]:
train_file = bz2.BZ2File('/content/drive/MyDrive/dataset/train.ft.txt.bz2')
test_file = bz2.BZ2File('/content/drive/MyDrive/dataset/test.ft.txt.bz2')

In [5]:
train_file_lines = train_file.readlines(100000)
test_file_lines = test_file.readlines(100000)
num_train = 10000
num_test = 10000


In [6]:
train_file_lines = [x.decode('utf-8') for x in train_file_lines]
test_file_lines = [x.decode('utf-8') for x in test_file_lines]

In [7]:
def remove_underscores(sentence):
    sentence= sentence.replace("_"," ")
    return sentence

In [8]:
spanish_words =  set(nltk.corpus.cess_esp.words())
english_words = set(nltk.corpus.words.words())
stopwords_english = stopwords.words('english')
stopwords_spanish = stopwords.words('spanish')
stopwords = stopwords_english + stopwords_spanish

def remove_stop_words(sentence):     #remove stop words and meaningless words
    new_sentence=""
    for word in sentence.split():
        word=word.lower()
        if word in english_words or word in spanish_words and word not in stopwords and word.isalpha():
            new_sentence=new_sentence+" "+word
    return new_sentence

In [9]:
def lemmatize_and_filter(sentence, min_word_length):        #to lemmatize words and loose the ones with length less than equal to 3.
    sent = ""
    for word in sentence.split():
        if len(lemmatizer.lemmatize(word))>=min_word_length:
            sent= sent+" "+lemmatizer.lemmatize(word)
    return(sent)

In [10]:
def split_label_review(dataa):
    #takes in list of raw data and returns list of string sentences and the list of their corresponding labels.
    labels = [int(re.findall("1|2", str(lines))[0]) for lines in dataa]
  
    
    
    reviews = [re.split("__label__[1|2]",str(lines))[1].strip().lower() for lines in dataa]
    return(reviews,labels)

In [11]:
def preprocessed_data(raw_text):        #takes in list of raw data and returns list of processed string sentences and the list of their corresponding labels.
    reviews = [lemmatize_and_filter(remove_stop_words(BeautifulSoup(re.sub(r'[^\w\s]|^https?:\/\/.*[\r\n]*|\d+', '', remove_underscores(str(lines)).strip().lower()), "lxml").text),3) for lines in raw_text]  
        
    return(reviews)

In [12]:
def convert_labels(negative_labels, positive_labels):
    negative_labels = [0 if x == '1' else 1 for x in negative_labels]
    positive_labels = [1 if x == '2' else 0 for x in negative_labels]
    final_labels = negative_labels + positive_labels
    return final_labels

In [13]:
def split_train_into_train_validate(split_):    #Takes in list of (raw or processed string) data and splits in train validation and returns in same format
    
    validation_ratio = abs(float(input("Enter the validation ratio between 0 to 1:\n")))
    if validation_ratio>1:
        validation_ratio = validation_ratio/100

    split = split_
    
    train_size = int(len(split)*(1-validation_ratio))
    
    random.shuffle(split)

    validate_samples= split[train_size:]

    split = split[:train_size]
    
    return(split,validate_samples)


In [14]:
def split_into_negative_positive(dataa):
    negative_reviews=[]      # list of all negative reviews
    positive_reviews=[]      # list of all positive reviews

    for lines in train_file_lines:
        lines= str(lines).lower()
        x=re.findall("1|2", lines)[0]
        if x=="1":
            negative_reviews.append(lines)
        elif x=="2":
            positive_reviews.append(lines)
    return(negative_reviews, positive_reviews)

In [15]:
train_file_lines,validation_file_lines = split_train_into_train_validate(train_file_lines)
negative_reviews, positive_reviews = split_into_negative_positive(train_file_lines)
negative_reviews,negative_labels = split_label_review(negative_reviews)
positive_reviews,positive_labels = split_label_review(positive_reviews)
negative_reviews = preprocessed_data(negative_reviews)
positive_reviews=preprocessed_data(positive_reviews)
#training_sentences, training_labels = split_label_review(train_file_lines)
final_labels = np.append(np.ones((len(positive_labels))), np.zeros((len(negative_labels))))
final_reviews = positive_reviews + negative_reviews

Enter the validation ratio between 0 to 1:
0.2


In [16]:
print(final_labels[50])
print(final_reviews[50])

1.0
 contemporary sure delight while this book the classic snow white for grown with some class act and rich and rewarding love story thoroughly reading and look forward more work


In [27]:
def build_sentence_final_label(raw_data):
    
    refined_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in raw_data]
    refined_labels = [1 if x.split(' ')[0] == '__label__2' else 0 for x in raw_data]
    refined_sentences = [x.split(' ', 1)[1][:-1].lower() for x in raw_data]
    
    labels_list = np.squeeze(refined_labels).tolist()
    freqs = []
    for sentence,labels in zip(refined_sentences,labels_list):
        
            pair = (sentence,labels)
            freqs += pair
    return freqs

print(build_sentence_final_label(train_file_lines[0:5]))

["my henry: i had henry back in the 70's. i don't remember him with a nose since it was chewed off, but he was well loved. he's long gone and i thought i was the only one who ever had a henry since nobody knew what i was talking about. i can't believe i've found him again!! nobody can replace my original worn henry, but i'm so excited to have his clone back!!", 1, 'a very good how-to manual.: if you are expecting a book full of glossy sharp pictures, this is not the one to buy.a lot of drawings and bw pictures that inform what they need to. this is a very good manual for those who want to venture in a timber frame project.', 1, "nice for the money: the first one i bought was defective. the dvd didn't work at all. i purchased it from electronics express and when i contacted them they were wonderful. they gave me an rma number right away, fedex picked it up the next day and a new one arrived 1 week later. this one is just fine. i really love the clarity of the picture. my only complaint 

In [18]:
negative_reviews_test,positive_reviews_test = split_into_negative_positive(test_file_lines)
negative_reviews_test,negative_labels_test = split_label_review(negative_reviews_test)
positive_reviews_test,positive_labels_test = split_label_review(positive_reviews_test)
negative_reviews_test = preprocessed_data(negative_reviews_test)
positive_reviews_test =preprocessed_data(positive_reviews_test)
final_labels_test = np.append(np.ones((len(positive_labels_test))), np.zeros((len(negative_labels_test))))
final_reviews_test = positive_reviews_test + negative_reviews_test

In [19]:
def custom_tokenize(text):
    if not text:
        print('The text to be tokenized is a None type. Defaulting to blank string.')
        text = ''
    return word_tokenize(text)




In [29]:
def build_freqs(refined_reviews, refined_labels):
    labels_list = np.squeeze(refined_labels).tolist()
    freqs = {}
    for sentence,labels in zip(refined_reviews,labels_list):
        for word in sentence.split():
            pair = (word,labels)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1    
    return freqs


freqs = build_freqs(final_reviews, final_labels)
print(freqs)

{('henry', 1.0): 13, ('had', 1.0): 18, ('back', 1.0): 5, ('the', 1.0): 348, ('dont', 1.0): 11, ('remember', 1.0): 2, ('him', 1.0): 12, ('with', 1.0): 45, ('nose', 1.0): 1, ('since', 1.0): 5, ('off', 1.0): 6, ('but', 1.0): 45, ('well', 1.0): 10, ('long', 1.0): 4, ('gone', 1.0): 4, ('and', 1.0): 200, ('thought', 1.0): 2, ('only', 1.0): 14, ('one', 1.0): 31, ('who', 1.0): 25, ('ever', 1.0): 7, ('nobody', 1.0): 2, ('knew', 1.0): 4, ('what', 1.0): 26, ('talking', 1.0): 2, ('about', 1.0): 14, ('cant', 1.0): 13, ('believe', 1.0): 1, ('found', 1.0): 5, ('again', 1.0): 10, ('can', 1.0): 20, ('replace', 1.0): 1, ('original', 1.0): 2, ('worn', 1.0): 1, ('excited', 1.0): 3, ('have', 1.0): 45, ('his', 1.0): 11, ('clone', 1.0): 1, ('very', 1.0): 23, ('good', 1.0): 26, ('manual', 1.0): 3, ('you', 1.0): 61, ('are', 1.0): 41, ('book', 1.0): 50, ('full', 1.0): 4, ('glossy', 1.0): 1, ('sharp', 1.0): 4, ('this', 1.0): 111, ('not', 1.0): 44, ('lot', 1.0): 12, ('that', 1.0): 55, ('inform', 1.0): 1, ('they',

In [30]:
words = Counter() #Dictionary that will map a word to the number of times it appeared in all the training sentences
for  sentence in final_reviews:
    #The sentences will be stored as a list of words/tokens
    
    for word in sentence.split():
        words.update([word.lower()]) #Converting all the words to lower case
        
    
        


print(words)

# Sorting the words according to the number of appearances, with the most common word being first
words = sorted(words, key=words.get, reverse=True)
# Adding padding and unknown to our vocabulary so that they will be assigned an index
words = ['_PAD','_UNK'] + words
# Dictionaries to store the word to index mappings and vice versa
word2idx = {o:i for i,o in enumerate(words)}
idx2word = {i:o for i,o in enumerate(words)}


Counter({'the': 744, 'and': 377, 'this': 250, 'for': 149, 'not': 132, 'that': 121, 'you': 120, 'but': 104, 'are': 103, 'have': 97, 'book': 87, 'with': 84, 'one': 77, 'from': 65, 'very': 64, 'they': 54, 'all': 53, 'what': 48, 'great': 48, 'game': 47, 'she': 47, 'like': 45, 'would': 42, 'them': 40, 'get': 39, 'just': 38, 'can': 36, 'good': 35, 'time': 35, 'out': 34, 'some': 34, 'your': 34, 'dont': 33, 'who': 33, 'old': 33, 'her': 33, 'read': 32, 'had': 31, 'when': 31, 'because': 31, 'there': 31, 'even': 30, 'these': 29, 'about': 28, 'more': 28, 'too': 27, 'will': 27, 'only': 26, 'love': 26, 'been': 26, 'much': 26, 'now': 26, 'other': 24, 'music': 24, 'work': 23, 'cant': 22, 'really': 22, 'got': 22, 'again': 21, 'his': 21, 'also': 21, 'movie': 21, 'product': 20, 'little': 20, 'never': 19, 'daughter': 19, 'any': 19, 'buy': 19, 'which': 19, 'over': 19, 'did': 19, 'bought': 18, 'could': 18, 'bad': 18, 'year': 18, 'fun': 18, 'make': 18, 'where': 18, 'than': 18, 'most': 18, 'well': 17, 'best':

In [ ]:
final_data = []

# loop through our selected words
for word in words:
    
    # initialize positive and negative counts
    pos = 0
    neg = 0
    
    # retrieve number of positive counts
    if (word, 1) in freqs:
        pos = freqs[(word, 1)]
        
    # retrieve number of negative counts
    if (word, 0) in freqs:
        neg = freqs[(word, 0)]
        
    # append the word counts to the table
    final_data.append([word, pos, neg])
    
print(final_data)



In [ ]:
fig, ax = plt.subplots(figsize = (8, 8))

# convert positive raw counts to logarithmic scale. we add 1 to avoid log(0)
x = np.log([x[1] + 1  for x in final_data])  

# do the same for the negative counts
y = np.log([x[2] + 1 for x in final_data]) 

# Plot a dot for each pair of words
ax.scatter(x, y)  

# assign axis labels
plt.xlabel("Log Positive count")
plt.ylabel("Log Negative count")

# Add the word as the label at the same position as you added the points just before
for i in range(0, len(final_data)):
    ax.annotate(final_data[i][0], (x[i], y[i]), fontsize=12)



In [ ]:
fig, ax = plt.subplots(figsize = (8, 8))

# convert positive raw counts to logarithmic scale. we add 1 to avoid log(0)
x = np.log([x[1] + 1  for x in final_data])  

# do the same for the negative counts
y = np.log([x[2] + 1 for x in final_data]) 

# Plot a dot for each pair of words
ax.scatter(x, y)  

# assign axis labels
plt.xlabel("Log Positive count")
plt.ylabel("Log Negative count")

# Add the word as the label at the same position as you added the points just before
for i in range(0, len(final_data)):
    ax.annotate(final_data[i][0], (x[i], y[i]), fontsize=12)

ax.plot([0, 9], [0, 9], color = 'red') # Plot the red line that divides the 2 areas.
plt.show()
plt.close()

In [ ]:
ig = plt.figure(figsize=(5, 5))

# labels for the two classes
labels = 'Positives', 'Negative'

# Sizes for each slide
sizes = [len(positive_reviews), len(negative_reviews)] 

# Declare pie chart, where the slices will be ordered and plotted counter-clockwise:
plt.pie(sizes, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)

# Equal aspect ratio ensures that pie is drawn as a circle.
plt.axis('equal')  

# Display the chart
plt.show()

In [ ]:
plot_1 = sns.countplot(final_labels)
plt.title('Train Labels distribution')
plt.show(plot_1)
plt.close()
plt.clf()

In [ ]:
plot_2 = sns.countplot(final_labels_test)
plt.title('Test Labels distribution')
plt.show(plot_2)
plt.close()
plt.clf()

In [ ]:
final_reviews_size =  len(final_reviews)
plot_3 = sns.displot(final_reviews)
plt.xlabel("#words in reviews")
plt.ylabel("Frequency")
plt.title("Word Frequency Distribution in Reviews")
plt.show(plot_3)
plt.close()
plt.clf()

In [ ]:
final_reviews_size =  len(test_file_lines)
train_sentences_size =  len( final_reviews)
train_label_len = pd.DataFrame({"labels": final_labels, "len": train_sentences_size})
print(train_label_len.head())


neg_mean_len = train_label_len.groupby('labels')['len'].mean().values[0]
pos_mean_len = train_label_len.groupby('labels')['len'].mean().values[1]

print(f"Negative mean length: {neg_mean_len:.2f}")
print(f"Positive mean length: {pos_mean_len:.2f}")
print(f"Mean Difference: {neg_mean_len-pos_mean_len:.2f}")
sns.catplot(x='labels', y='len', data=train_label_len, kind='box')
plt.xlabel("labels (0->negative, 1->positive)")
plt.ylabel("#words in reviews")
plt.title("Review Size Categorization")


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,LSTM

model1=  Sequential()
model1.add(LSTM(100,input_shape=(25,50),activation='relu'))
model1.add(Dense(50,activation='relu'))
model1.add(Dense(1,activation='sigmoid'))

model1.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
hist = model1.fit(final_reviews,final_labels,validation_split=0.1,epochs=6,batch_size=64,verbose=1)


model1.evaluate(final_reviews_test, final_labels_test, batch_size=128)

#model1.evaluate(final_reviews, final_labels, batch_size=128)

import matplotlib.pyplot as plt
loss_curve = hist.history['loss']
epoch_c = list(range(len(loss_curve)))

plt.xlabel('Epochs')
plt.ylabel('Loss value')
plt.plot(epoch_c,loss_curve,label='1 Hidden layer')
plt.show()

acc_curve = hist.history['acc']
epoch_c = list(range(len(loss_curve)))

plt.xlabel('Epochs')
plt.ylabel('Accuracy value')
plt.plot(epoch_c,acc_curve,label='1 Hidden layer')
plt.show()
